In [5]:
# Load dependencies
import pandas as pd

import spacy
from spacy.tokens import Doc
nlp = spacy.load('en_core_web_md')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import plotly

# Set sentiment extensions
sent_analyzer = SentimentIntensityAnalyzer()
def sentiment_scores(docx):
    return sent_analyzer.polarity_scores(docx.text)
Doc.set_extension("sentimenter",getter=sentiment_scores,force=True)

# Sentiment

In [6]:
# A helper function to get sentiment of a comment
def get_sentiment(text):
    return nlp(text)._.sentimenter['compound']

In [7]:
C1 = 'This is totoally unacceptable!'
C2 = 'The service of Air France is excellent.'
print(get_sentiment(C1))
print(get_sentiment(C2))

-0.5093
0.5719


In [21]:
# Run the function on our datasets

# df_seatguru['Sentiment'] = df_seatguru['Comment'].map(get_sentiment)
# df_seatguru.to_pickle('df_seatguru')
# df_seatguru.to_csv('df_seatguru.csv')

# Topic Modeling

In [8]:
# A preliminary topic model (Which definitely should be changed to something more advanced later on)
from gensim.parsing.preprocessing import preprocess_string
df_seatguru = pd.read_pickle('df_seatguru')

In [28]:
df_seatguru['Doc'] = df_seatguru['Comment'].map(preprocess_string)

In [30]:
import gensim
from gensim import corpora

In [31]:
dictionary = corpora.Dictionary(df_seatguru['Doc'])

In [36]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in df_seatguru['Doc']]

In [39]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100,
                chunksize=1000, passes=50)

In [40]:
lda_model.print_topics()

[(0,
  '0.066*"fly" + 0.059*"airlin" + 0.045*"plane" + 0.040*"avoid" + 0.038*"aircraft" + 0.033*"worst" + 0.020*"absolut" + 0.020*"cost" + 0.018*"jet" + 0.018*"cramp"'),
 (1,
  '0.143*"class" + 0.118*"busi" + 0.088*"seat" + 0.039*"flat" + 0.027*"bed" + 0.017*"sleep" + 0.016*"lie" + 0.015*"comfort" + 0.014*"best" + 0.014*"travel"'),
 (2,
  '0.161*"feel" + 0.085*"like" + 0.035*"singapor" + 0.034*"cramp" + 0.031*"number" + 0.030*"layout" + 0.030*"privat" + 0.028*"footrest" + 0.028*"felt" + 0.022*"version"'),
 (3,
  '0.083*"air" + 0.062*"cold" + 0.033*"hot" + 0.033*"poor" + 0.027*"unfortun" + 0.024*"accept" + 0.024*"lufthansa" + 0.024*"vent" + 0.024*"ride" + 0.022*"man"'),
 (4,
  '0.200*"seat" + 0.049*"uncomfort" + 0.047*"narrow" + 0.039*"reclin" + 0.025*"pitch" + 0.023*"inch" + 0.020*"knee" + 0.020*"hard" + 0.019*"tight" + 0.017*"tall"'),
 (5,
  '0.066*"tabl" + 0.066*"trai" + 0.062*"seat" + 0.058*"armrest" + 0.055*"arm" + 0.044*"rest" + 0.041*"power" + 0.025*"entertain" + 0.023*"usb" + 0.

In [45]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.214961 -0.112626       1        1  16.666599
13    -0.127621  0.192675       2        1  11.844498
8     -0.180879  0.076427       3        1  10.770675
1     -0.135010 -0.012359       4        1   6.579148
15    -0.166959 -0.193269       5        1   6.141716
4     -0.173395 -0.014038       6        1   5.605768
19    -0.162323 -0.103587       7        1   5.332251
6     -0.106659  0.191097       8        1   4.782888
18    -0.083755 -0.150685       9        1   4.613216
7      0.078132 -0.013289      10        1   3.605181
0      0.008400  0.235633      11        1   3.390963
5     -0.006703 -0.139914      12        1   3.373394
12    -0.060714  0.180014      13        1   3.283343
14     0.135831  0.040449      14        1   2.914320
11     0.103554 -0.114793      15        1   2.473487
9      0.190278 -0.075742      16        1   2.005787
17     0.237205 -0.033802      17        1   1.851225
10     0.254447 -0.024022      18        1   1.664995
3      0.226758 -0.011298      19        1   1.557316
2      0.184376  0.083130      20        1   1.543239, topic_info=    Category          Freq     Term         Total  loglift  logprob
27   Default   4873.000000      leg   4873.000000  30.0000  30.0000
25   Default  10467.000000   flight  10467.000000  29.0000  29.0000
5    Default  38007.000000     seat  38007.000000  28.0000  28.0000
127  Default   4831.000000  economi   4831.000000  27.0000  27.0000
29   Default   4271.000000     room   4271.000000  26.0000  26.0000
..       ...           ...      ...           ...      ...      ...
418  Topic20     94.458313    climb    240.120331   3.2383  -4.3327
216  Topic20    116.971458     aisl   2172.520264   1.2496  -4.1189
143  Topic20     90.269783   person   1549.323364   1.3285  -4.3780
196  Topic20     72.596024     look    936.120361   1.6145  -4.5959
390  Topic20     66.850883  privaci    458.954681   2.2448  -4.6784

[1139 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
831       7  0.982068      abc
105       7  0.979068     abil
1873     11  0.992483  abreast
272       8  0.160681  absolut
272      11  0.837097  absolut
...     ...       ...      ...
193      11  0.227221     year
1368      1  0.994428   yellow
1767     20  0.945532     your
39       14  0.981404      zrh
23       18  0.994191   zurich

[1751 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 14, 9, 2, 16, 5, 20, 7, 19, 8, 1, 6, 13, 15, 12, 10, 18, 11, 4, 3])